# CT example

아래의 코드는 
data는 각 환자 데이터들이 다른 폴더에 존재하고,
각 환자의 데이터폴더에는 image와 segmentation이 미리 정해진 이름으로 동일하게 존재하는 것을 가정합니다.

ex.

project1 --- patient001 --- image.nii.gz, seg.nii.gz
         --- patient002 --- image.nii.gz, seg.nii.gz
                ...


이 경우에 
path = "project1"이 들어가야 합니다. 
절대경로/상대경로에 대해 잘 이해가 되지 않는 경우에는 아래와 같이 project1의 상위폴더도 같이 넣어 줍니다.
path = "T:\python\data\project1"

path안에 있는 폴더이름을 환자의 id로 간주하여 처리하게 됩니다.

코드에서 아래에 해당하는 부분에서 파일이름이 image, seg가 아닌 경우 해당하는 이름으로 바꾸어 줍니다. 
image = sitk.ReadImage(path+"/"+patient_id[i]+"/"+"image.nii.gz")
label = sitk.ReadImage(path+"/"+patient_id[i]+"/"+"seg.nii.gz")

코드의 끝 부분을 보면 아래와 같은 코드가 있습니다. 
이 부분을 변경하여 내가 원하는 위치에 원하는 이름의 csv파일로 결과를 저장할 수 있습니다.
import csv
f = open('pyradiomics/results/msk_ct001_test.csv','w', newline="")


문의: yhnam83@gmail.com



In [2]:
# Download the zip file if it does not exist
import os, zipfile
import pandas as pd
import seaborn as sns
from six.moves import urllib
# Import some libraries
import SimpleITK as sitk
from radiomics import featureextractor

path, param을 본인의 폴더, 파일명으로 정확하게 입력해야 합니다. 
exampleCT.yaml파일은 pyradiomics에서 기본으로 제공하는 CT용 환경파일입니다.

In [8]:
# Load up the segmentations and extract the features
#path = "/content/drive/My Drive/Radiomics/Neuro/HYST2EDEMA"
path = "pyradiomics/data/msk_ct_001"
Input_folder = os.listdir(path)

params = os.path.join(os.getcwd(), 'pyradiomics/yaml', 'exampleCT.yaml')
extractor = featureextractor.RadiomicsFeatureExtractor(params)

patient_id = []
for i in range(0,len(Input_folder)):
    patient_id.append(Input_folder[i])

In [9]:
path+"/"+patient_id[i]+"/"+".nii.gz"

'pyradiomics/data/msk_ct_001/p001/.nii.gz'

In [10]:
features = {}
for i in range(0,len(patient_id)):
    print(patient_id[i])
    image = sitk.ReadImage(path+"/"+patient_id[i]+"/"+"image.dcm.nii.gz")
    label = sitk.ReadImage(path+"/"+patient_id[i]+"/"+"seg.nii.gz")
    features[i] = extractor.execute(image,label)

p001


In [13]:
feature_names0 = list(sorted(filter ( lambda k: k.startswith("original"), features[0])))  #key&value(dictionary)
feature_names1 = list(sorted(filter ( lambda k: k.startswith("wavelet"), features[0])))  #key&value(dictionary)
feature_names2 = list(sorted(filter ( lambda k: k.startswith("log"), features[0])))  #key&value(dictionary)

feature_names=feature_names0+feature_names1+feature_names2

def extractDigits(lst):
    res=[]
    for el in lst:
        sub=el.split(',')
        res.append(sub)
    return(res)

patient = extractDigits(patient_id)
#print(patient)

In [14]:
# Make a numpy array of all the values
import numpy as np

samples = np.zeros((len(patient_id),len(feature_names)))

for i in range(0,len(patient_id)):
    a = np.array([])
#  a = np.append(a,case_id)
    for feature_name in feature_names:
        a = np.append(a, features[i][feature_name])
  
    samples[i,:]=a


# May have NaNs
samples = np.nan_to_num(samples)

########
data=samples
#data=np.append(patient,data,axis=1)
data =np.append(patient,data,axis=1)

# save result to csv file
first_row=feature_names[:]
first_row.insert(0,'Patient ID')
#print(first_row)

import csv
f = open('pyradiomics/results/msk_ct001_test.csv','w', newline="")
wr = csv.writer(f)
wr.writerow(first_row)

for i in range(len(patient_id)):
    wr.writerow(data[i,:])

f.close()